# git revert

## 개요
`git revert`는 **특정 commit을 취소**하고, 취소된 상태로 **새로운 commit을 생성**하는 명령어dla

```bash
# 특정 commit 취소
git revert <취소할 commit_id>
```

**핵심 특징**:
- 취소할 commit을 반대로 되돌리는 **새로운 commit을 추가**
- 취소되는 commit도 **log에 기록이 남음** (히스토리 보존)
- **협업 환경(Remote Repository)**에서 안전하게 사용 가능

## git revert vs git reset

| 비교 항목 | git revert | git reset |
|-----------|------------|-----------|
| **인자** | 취소할 commit ID | 되돌아갈 commit ID |
| **히스토리** | 취소 기록이 남음 (새 commit 생성) | 취소된 commit이 log에서 삭제됨 |
| **협업 사용** | ✅ 안전하게 사용 가능 | ⚠️ 공동작업 시 피해야 함 |
| **동작 방식** | 새 commit으로 변경사항 취소 | HEAD를 이전 commit으로 이동 |

**협업 시 주의사항**:
- Remote Repository에서 공동작업 시 **반드시 `revert` 사용**
- `reset`은 히스토리를 삭제하므로 다른 개발자와 충돌 발생 가능

## 주요 옵션

### 1. `--no-commit` 옵션
자동 commit을 방지하고 Stage 단계에서 멈춤

```bash
# Revert 후 자동 commit 하지 않음
git revert --no-commit <commit_id>
```

**사용 예시**:
- Revert 후 추가 수정을 하고 싶을 때
- 여러 commit을 한꺼번에 revert 하고 한 번에 commit 하고 싶을 때

```bash
# 여러 commit을 revert 후 한 번에 commit
git revert --no-commit abc123
git revert --no-commit def456
# 추가 수정 작업...
git commit -m "Revert multiple commits with fixes"
```

## 범위 지정으로 여러 Commit 취소

특정 구간의 commit들을 한 번에 취소할 수 있음

```bash
# 특정 구간의 commit 모두 취소
git revert <시작_commit_id>..<끝_commit_id>
```

**범위 표기법**:
- `A..B`: A **직후**부터 B까지 (A 불포함, B 포함)
- `A^..B`: A부터 B까지 (A 포함, B 포함)
  - `^` (caret)은 parent commit을 의미

```bash
# 예시: commit ID 2 이후부터 현재까지 모두 취소
git revert <commit_id_2>..HEAD

# 예시: commit ID 2를 포함하여 현재까지 모두 취소
git revert <commit_id_2>^..HEAD
```

**왜 범위로 취소하는가?**
- 중간 commit만 취소하면 **conflict가 매우 쉽게 발생**
- 실무에서는 되돌리고자 하는 commit 이후 구간을 **모두 취소**하는 것이 안전

## Conflict 발생과 해결

### Conflict가 발생하는 이유
`git revert`는 취소하려는 commit의 **반대 작업**을 수행하는 새 commit을 만듦

**Conflict 발생 시나리오**:
1. Commit 2에서 `test.txt` 파일 추가
2. Commit 3에서 `test.txt` 파일 수정
3. `git revert <commit_2>` 실행
   - Git은 `test.txt`를 삭제하려고 함 (Commit 2 취소)
   - 하지만 Commit 3에서 해당 파일을 수정했음
   - **Conflict 발생!**

### Conflict 해결 방법

```bash
# 1. Conflict 발생
git revert abc123
# CONFLICT 메시지 출력

# 2. Conflict 해결 (3가지 방법 중 선택)

# 방법 1: 수동으로 파일 수정 후
git add <resolved_file>
git revert --continue

# 방법 2: 문제 파일 삭제 후
git rm <conflict_file>
git revert --continue

# 방법 3: Revert 취소
git revert --abort
```

**실제 예제**:

```bash
# Conflict 발생 시 메시지
CONFLICT (modify/delete): test01.txt deleted in parent of 63fa392 
and modified in HEAD. Version HEAD of test01.txt left in tree.
error: could not revert 63fa392... second commit

# 해결 방법 1: 파일 삭제로 해결
git rm test01.txt
git revert --continue

# Vim 에디터가 열리면 commit 메시지 확인
# ESC 누르고 :wq 입력하여 저장
```

## 실전 예제

```bash
# 1. 최근 commit 하나 취소
git revert HEAD

# 2. 특정 commit 취소
git revert abc1234

# 3. 여러 commit 동시에 취소 (자동 commit 방지)
git revert --no-commit HEAD~3..HEAD
git commit -m "Revert last 3 commits"

# 4. Commit ID 2부터 현재까지 모두 취소
git log  # commit ID 확인
git revert <commit_id_2>..HEAD

# 5. Conflict 발생 시 취소
git revert --abort
```

## 작업 흐름 요약

```bash
# 1. Log 확인
git log --oneline

# 2. 취소할 commit 선택 및 revert
git revert <commit_id>

# 3-1. Conflict 없으면 자동으로 commit 완료
# 3-2. Conflict 발생 시
#      - 파일 수정/삭제로 해결
#      - git add 또는 git rm
#      - git revert --continue
#      - Vim에서 commit 메시지 확인 후 저장

# 4. 결과 확인
git log --oneline
```

## 주의사항

⚠️ **Conflict 빈번 발생**
- Revert 하려는 commit 이후에 관련 파일이 수정되었다면 conflict 발생 가능성 높음

⚠️ **범위 지정 권장**
- 중간 commit만 취소하기보다는 해당 시점 이후를 모두 취소하는 것이 안전

✅ **협업 환경에서 필수**
- Remote repository에 push된 commit은 `reset` 대신 **반드시 `revert` 사용**

---
**출처**: [ds31x.tistory.com/157](https://ds31x.tistory.com/157)
